best_parameters_fit_C1case7，病例阈值设为了5，

OEV_case[i, t] = max(0.0001, obs_ave ** 2 / 600000)

xita=1.2

ensemble = 100

In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import math
import random
import os
import time
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors as mcolors
from matplotlib.pyplot import MultipleLocator
#从pyplot导入MultipleLocator类，这个类用于设置刻度间隔
import matplotlib.ticker as ticker

C:\Users\Keli\AppData\Roaming\Python\Python37\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
parafit = pd.read_csv("best_parameters.csv")
# parafit = pd.read_csv("best_parameters_fit_C1case7.csv")
#R0,mu,theta,Te,alpha,Ti
parafit = np.array(parafit).astype(np.float64)

In [3]:
raw_population = pd.read_csv('usmainpopchange_2019.csv')
initial_population = raw_population['pop'].to_numpy().astype(np.float64)

In [4]:
#2020年1月1日至2020年8月2日每日新增病例
observed_daily_infectious_count = pd.read_csv("covid19_dailyconfirmed_USmainstate1-8.csv",index_col=0)
# observed_daily_infectious_count.drop('Province_State', axis=1) 
raw_incidence = observed_daily_infectious_count.to_numpy()[:, 5:215].astype(np.float64)
# raw_incidence = observed_daily_infectious_count.to_numpy()[:215, 1:]
# raw_incidence = np.transpose(raw_incidence).astype(np.float32)

In [5]:
raw_mobility = pd.read_csv("USdaily_mainflows1-8ODdiff.csv") 
daily_mobility_matrices = []
for i in range(1, 216):
    day_mobility = raw_mobility[raw_mobility['day'] == i]
#     print(day_mobility)
  
     #Make a matrix of daily mobilities.
    z = pd.crosstab(
      day_mobility.Origin, 
      day_mobility.Destination, 
      values=day_mobility['visitor_flows'], aggfunc='sum', dropna=False)
  
    # Include every city, even if there are no rows for some in the raw data on
    # some day.  This uses the sort order of `raw_population`.
    z = z.reindex(index=raw_population['RegionName'], columns=raw_population['RegionName'], 
                fill_value=0)
    # Finally, fill any missing entries with 0. This means no mobility.
    z = z.fillna(0)
    daily_mobility_matrices.append(z.to_numpy())
    
mobility_matrix_over_time = np.stack(daily_mobility_matrices, axis=-1).astype(np.float64)

In [6]:
mobility_matrix_over_time = mobility_matrix_over_time[:,:,5:215]

In [7]:
#延迟天数，4-8月每个月的delay都不同
delaypara = pd.read_csv("delaypara.csv")
delaypara = np.array(delaypara).astype(np.float64)

In [8]:
#干预影响
effect = pd.read_csv("US_state_effect_flowall_daily.csv")
effect.drop('Date', axis=1) 
total_effect = effect.to_numpy()[:, 1:]
total_effect = np.transpose(total_effect).astype(np.float64)/100

In [9]:
print('Mobility Matrix over time should have shape (49, 49, 210): {}'.format(
    mobility_matrix_over_time.shape))
print('Observed Infectious should have shape (49, 210): {}'.format(
    raw_incidence.shape))
print('Initial population should have shape (49): {}'.format(
    initial_population.shape))

print('Total effect should have shape (49, 210): {}'.format(
    total_effect.shape))

Mobility Matrix over time should have shape (49, 49, 210): (49, 49, 210)
Observed Infectious should have shape (49, 210): (49, 210)
Initial population should have shape (49): (49,)
Total effect should have shape (49, 210): (49, 210)


In [10]:
def initialzepara(incidence, num_ens, parafit, pop):
    # initialize ensemble members，先限定参数范围
    # para: R0,Te,p,a,theta,Ti1,Ti2,...,Ti49,beta1
    # R0
    R0min = 1
    R0max = 5
    # latency period
    Temin = 2
    Temax = 6
    # infectious period
    Timin = 2
    Timax = 9
    # relative transmissibility
    pmin = 0.2
    pmax = 1
    # asymptomatic rate
    amin = 0.02
    amax = 1
    # movement factor
    thetamin = 1
    thetamax = 2

    num_loc = len(incidence)
    Timap = [i + 4 for i in range(num_loc)]
    amap = [i + 4 + num_loc for i in range(num_loc)]
    # 导入各州的人口密度
    PD = np.log10(pop)


    # 参数的最大最小值组合
    # R0,Te,p,theta,Ti1,...,Ti49,a1,...,a49
    paramin4 = [R0min, Temin, pmin, thetamin]
    Timina = np.ones(num_loc) * Timin
    amina = np.ones(num_loc) * amin
    paramin = np.hstack((paramin4, Timina, amina))
    paramax4 = [R0max, Temax, pmax, thetamax]
    Timaxa = np.ones(num_loc) * Timax
    amaxa = np.ones(num_loc) * amax
    paramax = np.hstack((paramax4, Timaxa, amaxa))

    para = np.zeros((len(paramin), num_ens))

    # 参数拟合
    # R0
    para[0, :] = np.median(parafit[0, :]) * np.ones(num_ens)
    # Te
    para[1, :] = np.median(parafit[3, :]) * np.ones(num_ens)
    # p
    para[2, :] = np.median(parafit[1, :]) * np.ones(num_ens)
    # theta
    para[3, :] = np.median(parafit[2, :]) * np.ones(num_ens)

    for i in range(0, num_loc):
        # Ti
        para[Timap[i], :] = parafit[5, np.ceil(np.random.rand(num_ens) * np.size(parafit, 1)).astype(int) - 1]
        # a
        para[amap[i], :] = parafit[4, np.ceil(np.random.rand(num_ens) * np.size(parafit, 1)).astype(int) - 1]

    return para, paramax, paramin, Timap, amap


In [11]:
def initialize(incidence,num_ens,pop):
#对S,E,I,R和病例初始种子点Seedc初始化
    num_loc = len(incidence)

    # S，易感者S先假定为当然人口数
    S = pop.reshape(num_loc,1)* np.ones(num_ens)
    # E，潜伏者初始化为0
    E = np.zeros((num_loc,num_ens))
    # Is，有症状患者初始化为0
    Is = np.zeros((num_loc,num_ens))
    # Ia，无症状患者初始化为0
    Ia = np.zeros((num_loc,num_ens))

    # 模拟天数
    num_times = np.size(incidence,1)

    # 给各个地区设置初始病例种子
    Seedc = np.zeros((num_loc,num_times))

    # 给各地区病例排序
    # 计算累积病例
    reported = np.sum(incidence, 1)


    for i in range(0,len(reported)):
        seedloc = i
        temp = incidence[seedloc, :]
        #如果病例小于等于1，below为1，其他为0
        below = (np.array(temp) <= 1).astype(int)
        #定义起始点列表
        startpoints=[]
        #设置计数初始值
        cnt = 0
        last = 1
#         print("state",i)
        #选择初始种子点点列表
        #leng(below)即为天数
        for i in range(0, len(below)):
        # 如果below为1，即该天的每日新增小于等于1，则cnt+1
            if ( below[i] == 1):
                cnt = cnt+1
            else:
            # 否则，如果cnt＞=7时，startpoints数组扩展加入last，last重新赋值给i，即为该天的天数；否则below大于1时，cnt仍为0
            # cnt＞=7意味着该区域至少有7天新增病例都小于等于1,
                if (cnt >= 7):
                    startpoints.append(last)
                    last = i
                cnt = 0

        for j in range(0,len(startpoints)):
            index = np.array(np.nonzero(temp >= 5))
            # 保留病例大于等于5，且至少7天每日新增小于等于1之后该地区的病例数
            index = index[index >= startpoints[j]]
            # 如果index非空，前8天的初始种子即为前五天的总病例数（初始点选择假设平均报告延迟为8天）
            if index is not None:
                # first reporting with at least 5 cases
                T0 = index[0]
#                 print("T0",T0)
                #首次报告新增病例大于等于5例之后及后四天共五天报告的病例总数
                c = np.sum(temp[T0:min(T0+6,num_times)])
                #令报告日期的前八天的初始病例数为病例总和
                Seedc[seedloc,max(0,T0-8)] = c
#                 print("Seedc[seedloc,max(0,T0-8)]",Seedc[seedloc,max(0,T0-8)])
    return S, E, Is, Ia, Seedc

In [12]:
def seeding(S, E, Is, Ia, pop,Seedc, t):
    num_loc = len(Seedc)
    #ensemble,一般取10个
    num_ens = np.size(S,1)


    for i in range(0,num_loc):
        seedloc = i
        # 在初始日期的前9天从均匀分布[0,12C]和[0,10C],[0,2C]中随机抽取了E和Ia,Is,C为T0到T0+4这五天报告的病例总数
        #选取规则参考Peisen Nature 2021
        #由于只有T0的前八天有初始病例之，因此seedE，seedIa，seedIs也都在该天，其他天初始值均为0
        
        seedE = np.round(np.random.rand(num_ens) * 12 * Seedc[i, t])

        seedIa = np.round(np.random.rand(num_ens) * 10 * Seedc[i, t])

        seedIs = np.round(np.random.rand(num_ens) * 2 * Seedc[i, t])

        #计算t时刻种子点所在位置产生的交互


        if (Seedc[i,t] > 0):
            #乘交互影响
            E[i, :] = np.round(seedE * S[i,:] / pop[i])
            Ia[i, :] = np.round(seedIa * S[i,:] / pop[i])
            Is[i, :] = np.round(seedIs * S[i,:] / pop[i])
            S[i, :]= pop[i]-E[i, :]-Is[i, :]-Ia[i, :]

    return S, E, Is, Ia

In [13]:
def checkbound(S, E, Is, Ia):
    # check variable bound
    for i in range(0, np.size(S, 1)):
        S[S[:, i] < 0, i] = 0
        E[E[:, i] < 0, i] = 0
        Is[Is[:, i] < 0, i] = 0
        Ia[Ia[:, i] < 0, i] = 0

    return S, E, Is, Ia

In [14]:
def SEIR(S0, E0, Is0, Ia0, N,pop, para, M, Timap, amap, effect):
    # 输入的应该也是M(:,:,t)
    # the metapopulation SEIR model
    S = S0
    E = E0
    Is = Is0
    Ia = Ia0
    para = para
    Timap = Timap
    amap = amap

    # figure out dimensions

    num_loc = len(pop)

    # associate outputs
    dailyIs = np.zeros(num_loc)
    dailyIa = np.zeros(num_loc)

    # 参数赋值
    R0 = para[0]
    Te = para[1]
    p = para[2]
    theta = para[3]
#     print('R0',R0)
    
    # difine new State
    newS = np.zeros(num_loc)
    newE = np.zeros(num_loc)
    newIs = np.zeros(num_loc)
    newIa = np.zeros(num_loc)
    newR = np.zeros(num_loc)
    R = np.zeros(num_loc)
    
    #初始化差值
    ESenterD = np.zeros(num_loc) 
    ESleftD = np.zeros(num_loc) 
    EEenterD = np.zeros(num_loc) 
    EEleftD = np.zeros(num_loc) 
    EIaenterD = np.zeros(num_loc) 
    EIaleftD = np.zeros(num_loc) 
    EIaenterD = np.zeros(num_loc) 
    EexprD = np.zeros(num_loc) 
    EexpaD = np.zeros(num_loc) 
    EinfrD = np.zeros(num_loc) 
    EinfaD = np.zeros(num_loc) 
    ErecrD = np.zeros(num_loc) 
    ErecaD = np.zeros(num_loc) 
     
    
    
    NewN = N.copy()   
#     print(N,"NewN")

    # assgn new S, E, Ia, Is, R
    for i in range(0, num_loc):
        R[i] = max(N[i] - S[i] - E[i] - Is[i] - Ia[i], 0)
        newS[i] = S[i]
        newE[i] = E[i]
        newIs[i] = Is[i]
        newIa[i] = Ia[i]
        newR[i] = R[i]

    # total outgoing population
    popleft = np.zeros(num_loc)
    # enter S, E, Ia
    Sent = np.zeros(num_loc)
    Eent = np.zeros(num_loc)
    Iaent = np.zeros(num_loc)  
    
    # compute Sent, Eent and Iaent,计算输入S,E,Ia
    for i in range(0, num_loc):
        for j in range(0, num_loc):
            Sent[i] = Sent[i] + M[j, i] * S[j] / (NewN[j] - Is[j])
            Eent[i] = Eent[i] + M[j, i] * E[j] / (NewN[j] - Is[j])
            Iaent[i] = Iaent[i] + M[j, i] * Ia[j] / (NewN[j] - Is[j])
    
    # compute popleft,记录从i输出的总人口
    popleft = np.sum(M,0)
    
    # compute population
    NewN = NewN - theta * np.sum(M,0) + theta * np.sum(M,1)
#     print("NewN",np.sum(NewN))
#     print("theta",theta)
    
    # States can not fall below this fraction of people
    minimum_city_fraction = 0.6
    for i in range(0, num_loc):
        if (NewN[i] < minimum_city_fraction * pop[i]):
            NewN[i] = minimum_city_fraction * pop[i]
#     print("NewN",np.sum(NewN))    
#     print("p",p)  

#     print("Te",Te)

    # compute for each subpopulation
    # 引入直接和间接效应的影响
    for i in range(0, num_loc):
        beta = R0 / para[Timap[i]]
        alpha = para[amap[i]]
        # new exposed due to reported cases
        Eexpr = beta * S[i] * (1 - effect[i]) * Is[i] / NewN[i]
        # new exposed due to unreported cases
        Eexpa = p * beta * (1 - effect[i]) * S[i] * Ia[i] / NewN[i]
        # new reported cases
        Einfr = alpha * E[i] / Te
        # new unreported cases
        Einfa = (1 - alpha) * E[i] / Te
        # new recovery of reported cases
        Erecr = Is[i] / para[Timap[i]]
        # new recovery of unreported cases
        Ereca = Ia[i] / para[Timap[i]]
        

        

        # incoming S
        ESenter = theta * (NewN[i] - Is[i]) / NewN[i] * Sent[i]
        # outgoing S
        ESleft = theta * S[i] / (NewN[i] - Is[i]) * popleft[i]

        # incoming E
        EEenter = theta * (NewN[i] - Is[i]) / NewN[i] * Eent[i]
        # outgoing E
        EEleft = theta * E[i] / (NewN[i] - Is[i]) * popleft[i]
        # incoming Ia
        EIaenter = theta * (NewN[i] - Is[i]) / NewN[i] * Iaent[i]
        # outgoing Ia
        EIaleft = theta * Ia[i] / (NewN[i] - Is[i]) * popleft[i]
        


        ESenter = max(ESenter, 0)
        ESleft = max(ESleft, 0)
        EEenter = max(EEenter, 0)
        EEleft = max(EEleft, 0)
        EIaenter = max(EIaenter, 0)
        EIaleft = max(EIaleft, 0)
        Eexpr = max(Eexpr, 0)
        Eexpa = max(Eexpa, 0)
        Einfr = max(Einfr, 0)
        Einfa = max(Einfa, 0)
        Erecr = max(Erecr, 0)
        Ereca = max(Ereca, 0)
        
                
        
        # stochastic version
        ESenterp = np.random.poisson(ESenter)
        ESleftp = np.random.poisson(ESleft)
        EEenterp = np.random.poisson(EEenter)
        EEleftp = np.random.poisson(EEleft)
        EIaenterp = np.random.poisson(EIaenter)
        EIaleftp = np.random.poisson(EIaleft)
        Eexprp = np.random.poisson(Eexpr)
        Eexpap = np.random.poisson(Eexpa)
        Einfrp = np.random.poisson(Einfr)
        Einfap = np.random.poisson(Einfa)
        Erecrp = np.random.poisson(Erecr)
        Erecap = np.random.poisson(Ereca)
        
        ESenterD[i] = ESenterp - ESenter
        ESleftD[i] = ESleftp - ESleft
        EEenterD[i] = EEenterp - EEenter
        EEleftD[i] = EEleftp - EEleft
        EIaenterD[i] = EIaenterp - EIaenter
        EIaleftD[i] = EIaleftp - EIaleft
        EIaenterD[i] = EIaenterp - EIaenter
        EexprD[i] = Eexprp - Eexpr
        EexpaD[i] = Eexpap - Eexpa
        EinfrD[i] = Einfrp - Einfr
        EinfaD[i] = Einfap - Einfa
        ErecrD[i] = Erecrp - Erecr
        ErecaD[i] = Erecap - Ereca

            
        
        # 更新新病例
        newS[i] = np.round(max((newS[i] - Eexprp- Eexpap + ESenterp - ESleftp), 0))
        newE[i] = np.round(max((newE[i] + Eexprp + Eexpap - Einfrp - Einfap + EEenterp - EEleftp), 0))
        newIs[i] = np.round(max((newIs[i] + Einfrp - Erecrp), 0))
        newIa[i] = np.round(max((newIa[i] + Einfap - Erecap + EIaenterp - EIaleftp), 0))
        dailyIs[i] = np.round(max((dailyIs[i] + Einfrp), 0))
        dailyIa[i] = np.round(max((dailyIa[i] + Einfap), 0))
        newR[i] = np.round(max((NewN[i] - newE[i] - newIs[i] - newIa[i] - newS[i]), 0))
#         if(newE[i]>0):
#             print("state",i)
#             print("newE[i]_later",newE[i])     
    
    return newS, newE, newIs, newIa, dailyIs, dailyIa, NewN,ESenterD,ESleftD,EEenterD,EEleftD,EIaenterD,EIaleftD,EexprD,EexpaD,EinfrD,EinfaD,ErecrD,ErecaD



In [15]:
# print('Starting inference.')

start_time = time.time()
population = initial_population
np.random.seed(1234)
########################
FlowM = mobility_matrix_over_time.copy()
num_loc = len(FlowM)
num_ens = 100
num_times = np.size(raw_incidence, 1)
obs_case = np.zeros((num_loc, num_times))
#smooth the data: 7 day moving average，每日新增病例7天平滑
for i in range(0, num_loc):
    for t in range(0, num_times):
        if (t+3 < num_times):
            obs_case[i,t] = np.mean(raw_incidence[i,max(1,t-3):min(t+3,num_times)])
        else:
            obs_case[i, t] = np.mean(raw_incidence[i, max(1, num_times - 7):num_times-1])

# number of days to keep track of reported cases
T = num_times
Tstart = 0
# set OEV, observation error variance
OEV_case = np.zeros((num_loc, num_times))
for i in range(0, num_loc):
    for t in range(0, num_times):
        obs_ave = np.mean(raw_incidence[i, max(0,t-6):t])
        OEV_case[i, t] = max(0.0001, obs_ave ** 2 / 50)


# initialize model
S, E, Is, Ia, Seedc = initialize(raw_incidence, num_ens, population)

# records of reported cases
obs_temp = np.zeros((num_loc, num_ens, T))
# initialize parameters
para, paramax, paramin, Timap, amap = initialzepara(raw_incidence, num_ens, parafit, population)
para_ori = para

# inflate variables and parameters to get larger spread
# 膨胀变量和参数以获得更大的传播范围，膨胀参数的设置一般是Kalman filer中要求的
# 例子中设置两个inflate variables，常见的是设置一个
lambda1 = 2
para = np.mean(para, 1).reshape(len(para), 1) * np.ones(num_ens) + lambda1 * (
            para - np.mean(para, 1).reshape(len(para), 1) * np.ones(num_ens))
# para = checkbound_para(para, paramax, paramin)
for i in range(0, len(para)):
    para[i, para[i, :] < paramin[i]] = paramin[i] * (1 + 0.2 * np.random.rand(np.sum(para[i, :] < paramin[i])))
    para[i, para[i, :] > paramax[i]] = paramax[i] * (1 - 0.2 * np.random.rand(np.sum(para[i, :] > paramax[i])))

E = np.mean(E, 1).reshape(num_loc, 1) * np.ones(num_ens) + lambda1 * (
            E - np.mean(E, 1).reshape(num_loc, 1) * np.ones(num_ens))
Is = np.mean(Is, 1).reshape(num_loc, 1) * np.ones(num_ens) + lambda1 * (
            Is - np.mean(Is, 1).reshape(num_loc, 1) * np.ones(num_ens))
Ia = np.mean(Ia, 1).reshape(num_loc, 1) * np.ones(num_ens) + lambda1 * (
            Ia - np.mean(Ia, 1).reshape(num_loc, 1) * np.ones(num_ens))
S, E, Is, Ia = checkbound(S, E, Is, Ia)


# fix R0,p,theta Te, 所有州都相同，且不随时间变化

# R0,p,theta Te
# R0
para[0, :] = parafit[0, 0:num_ens]
# Te
para[1, :] = parafit[3, 0:num_ens]
# miu
para[2, :] = parafit[1, 0:num_ens]
# theta
para[3, :] = parafit[2, 0:num_ens]
# get ensemble spread of parameters
parastd = np.std(para, axis=1)

###################################
# posterior reported infection
dailyIs_post_rec = np.zeros((num_loc, num_ens, T))
# posterior unreported infection
dailyIa_post_rec = np.zeros((num_loc, num_ens, T))
# initialize poseteriors (percentage of population)
S_post = np.zeros((num_loc, num_times, num_ens))
E_post = np.zeros((num_loc, num_times, num_ens))
Is_post = np.zeros((num_loc, num_times, num_ens))
Ia_post = np.zeros((num_loc, num_times, num_ens))
# initialize cumulative reported and unreported infections
cumu_dailyIs_post = np.zeros((num_loc, num_ens))
cumu_dailyIa_post = np.zeros((num_loc, num_ens))
# 人口*时间，为了后续记录人口变化，构建49*215的人口矩阵
popNew = initial_population.reshape(num_loc, 1) * np.ones(num_times)



obs_temp1_rec = np.zeros((num_loc, num_ens, T))
###################################
# inflation in EAKF to avoid divergence
lambda2 = 1.2
# number of parameters
num_para = len(para)
# posterior parameters
para_post = np.zeros((num_para, num_ens, num_times))
para_prior = np.zeros((num_para, num_ens, num_times))
post_mean_rec = np.zeros((num_loc,num_times)).astype(float)
E_A_rec = np.zeros((num_loc,num_times)).astype(float)
Is_A_rec = np.zeros((num_loc,num_times)).astype(float)
Ia_A_rec = np.zeros((num_loc,num_times)).astype(float)
dailyIs_A_rec = np.zeros((num_loc,num_times)).astype(float)
dailyIa_A_rec = np.zeros((num_loc,num_times)).astype(float)
a_A_rec = np.zeros((num_loc,num_times)).astype(float)
Ti_A_rec = np.zeros((num_loc,num_times)).astype(float)
rnds_delay = []
rnds_delay_tmp = []

S_prior = np.zeros((num_loc, num_ens, num_times))
E_prior = np.zeros((num_loc, num_ens, num_times))
Is_prior = np.zeros((num_loc, num_ens, num_times))
Ia_prior = np.zeros((num_loc, num_ens, num_times))
S_seed = np.zeros((num_loc, num_ens, num_times))
E_seed = np.zeros((num_loc, num_ens, num_times))
Is_seed = np.zeros((num_loc, num_ens, num_times))
Ia_seed = np.zeros((num_loc, num_ens, num_times))

SenterD = np.zeros((num_loc, num_ens, num_times))
SleftD = np.zeros((num_loc, num_ens, num_times))
EenterD = np.zeros((num_loc, num_ens, num_times))
EleftD = np.zeros((num_loc, num_ens, num_times))
IaenterD = np.zeros((num_loc, num_ens, num_times))
IaleftD = np.zeros((num_loc, num_ens, num_times))
exprD = np.zeros((num_loc, num_ens, num_times))
expaD = np.zeros((num_loc, num_ens, num_times))
infrD = np.zeros((num_loc, num_ens, num_times))
infaD = np.zeros((num_loc, num_ens, num_times))
recrD = np.zeros((num_loc, num_ens, num_times))
recaD = np.zeros((num_loc, num_ens, num_times))

#备份
SenterD_temp = SenterD.copy()
SleftD_temp = SleftD.copy()
EenterD_temp = EenterD.copy()
EleftD_temp = EleftD.copy()
IaenterD_temp = IaenterD.copy()
IaleftD_temp = IaleftD.copy()
exprD_temp = exprD.copy()
expaD_temp = expaD.copy()
infrD_temp = infrD.copy()
infaD_temp = infaD.copy()
recrD_temp = recrD.copy()
recaD_temp = recaD.copy()




# 开始EAKF模拟，从2020年1月1日至8月2日
for t in range(0,num_times):
    print("模拟天数", t)

    locs = range(0, num_loc)

    # fix R0,p,theta Te，然后重新修正参数
    # R0
    para[0, :] = parafit[0, 0:num_ens]
    # Te
    para[1, :] = parafit[3, 0:num_ens]
    # p
    para[2, :] = parafit[1, 0:num_ens]
    # theta
    para[3, :] = parafit[2, 0:num_ens]
    
    
    #上一天的后验
    if(t>0):
        S_prior[:,:,t] = S 
        E_prior[:,:,t] = E
        Is_prior[:,:,t] = Is
        Ia_prior[:,:,t] = Ia 

    

    # seeding，种子点采样
    if (t < np.size(Seedc, 1)):
        S, E, Is, Ia = seeding(S, E, Is, Ia, popNew[:,t], Seedc, t)
        
        
    # re-initialize Timap
    for i in range(0, num_loc):
        if (Seedc[i, t] > 0):
            para[Timap[i], :] = para_ori[Timap[i], :]
            
    #seed之后的值
    S_seed[:,:,t] = S 
    E_seed[:,:,t] = E
    Is_seed[:,:,t] = Is
    Ia_seed[:,:,t] = Ia 
#     print('S',np.sum(S))
#     print('E',np.sum(E))
#     print('Is',np.sum(Is))
#     print('Ia',np.sum(Ia))

    #     para = checkbound_para(para, paramax, paramin)
    for i in range(0, len(para)):
        para[i, para[i, :] < paramin[i]] = paramin[i] * (1 + 0.2 * np.random.rand(np.sum(para[i, :] < paramin[i])))
        para[i, para[i, :] > paramax[i]] = paramax[i] * (1 - 0.2 * np.random.rand(np.sum(para[i, :] > paramax[i])))
    ###############
    
    para_prior[:, :, t] = para

    S_temp = S.copy()
    E_temp = E.copy()
    Is_temp = Is.copy()
    Ia_temp = Ia.copy()
    N_temp = population.copy()
    # integrate forward one step向前整合一步
    dailyIs_prior = np.zeros((num_loc, num_ens))
    dailyIa_prior = np.zeros((num_loc, num_ens))


    
    # run for each ensemble member，
    # 将前一天得到的模拟值作为先验
    
    for k in range(0, num_ens):
        S[:, k], E[:, k], Is[:, k], Ia[:, k], dailyIs_temp, dailyIa_temp, newN,SenterD[:, k, t],SleftD[:, k, t]\
        ,EenterD[:, k, t],EleftD[:, k, t],IaenterD[:, k, t], IaleftD[:, k, t],exprD[:, k, t],expaD[:, k, t]\
        ,infrD[:, k, t],infaD[:, k, t],recrD[:, k, t],recaD[:, k, t] = SEIR(S[:, k], E[:, k], Is[:, k],Ia[:, k]\
        ,popNew[:, t],population,para[:, k], FlowM[:, :, t],Timap, amap, total_effect[:, t])
#         if(t==46):

            
        dailyIs_prior[:, k] = dailyIs_temp
        dailyIa_prior[:, k] = dailyIa_temp

        #更新因交互导致的人口变化
        if (t + 1 < num_times):
            popNew[:, t + 1] = newN


    ###############
    # integrate forward for 9 days, prepare for observation
    # 向前整合9天，为了再次考虑病例延迟，使用前9天的发生率来约束当前模型变量和参数
    Tproj = 9
    obs_temp1 = obs_temp.copy()
    for t1 in range(t, t + Tproj):
        # run for each ensemble member
        if t1 >= num_times:
            t1 = num_times - 1
        for j in range(0, num_ens):
            S_temp[:, j], E_temp[:, j], Is_temp[:, j], Ia_temp[:, j], dailyIs_temp, dailyIa_temp, N_temp,\
            SenterD_temp[:, j, t1],SleftD_temp[:, j, t1],EenterD_temp[:, j, t1],EleftD_temp[:, j, t1],\
            IaenterD_temp[:, j, t1], IaleftD_temp[:, j, t1],exprD_temp[:, j, t1],expaD_temp[:, j, t1],\
            infrD_temp[:, j, t1],infaD_temp[:, j, t1],recrD_temp[:, j, t1],recaD_temp[:, j, t1] = SEIR(S_temp[:, j], \
            E_temp[:, j], Is_temp[:, j], Ia_temp[:, j], popNew[:, t1],population,para[:, j], FlowM[:, :, t1], Timap, amap, total_effect[:, t1])
            if (t1 + 1 < num_times):
                popNew[:, t1 + 1] = N_temp
            # reporting delay
            if t1 < 116:
                monthnum = 0
            if ((t1 >= 116) & (t1 < 147)):
                monthnum = 1
            if ((t1 >= 147) & (t1 < 177)):
                monthnum = 2
            if ((t1 >= 177) & (t1 < 208)):
                monthnum = 3
            if ((t1 >= 208) & (t1 < 210)):
                monthnum = 4
                
            a = delaypara[monthnum, 0]
            b = delaypara[monthnum, 1]

            for m in range(0, num_loc):
                # 估计的有记录的病例数
                inci = dailyIs_temp[m].astype(int)
                if(inci > 0):
                    rnd = np.ceil(np.random.gamma(a,b,inci)).astype(int).tolist()
#                     index = np.array([m,t1]).astype(int).tolist()
#                     rnds_index = index + rnd
#                     rnds_delay.append(rnds_index.copy())
                    # 对于选中的延迟，如果某天当天日期加延迟仍然小于模拟的最后一天，则该天的观察值+1
                    for h in range(0, len(rnd)):
                        if (t1 + rnd[h] < T):
                            obs_temp1[m, j, t1 + rnd[h]] = obs_temp1[m, j, t1 + rnd[h]] + 1

    ###############
    # 令t1为9天后的时间，且小于最后一天
    t1 = t + Tproj
    t1 = min(t1, num_times - 1)
    # 在t1的观察，作为先验
    obs_ens = obs_temp1[:, :, t1].copy()
    obs_temp1_rec[:,:,t] = obs_temp1[:, :, t1]
    para_prior[:, :, t] = para
#     print("t1",t1)
# #     print("obs_ens",np.sum(obs_ens))
#     print("obs_ens",np.sum(obs_temp1_rec[:,:,t]))
    
    # loop through local observations，通过当地的观察值进行循环
    for l in range(0, num_loc):
        # 如果前t1天的累计病例大于0
        if (np.sum(obs_case[l, 0:t1 + 1]) > 0):
            # Get the variance of the ensemble，获得ensemble的方差,观测值方差和预定的观察方差
            obs_var = OEV_case[l, t1]
            prior_var = np.var(obs_ens[l, :])

            # 根据先验和观测方差计算后验方差
            post_var = prior_var * obs_var / (prior_var + obs_var)
            # 如果先验方差为0，则令他们为0.001
            if (prior_var == 0):
                post_var = 1e-3
                prior_var = 1e-3
                
#             print("obs_var",np.sum(obs_var))
#             print("prior_var",np.sum(prior_var))
#             print("post_var",np.sum(post_var))           
            
            # 先验均值为十次ensemble的均值
            prior_mean = np.mean(obs_ens[l, :])
            # 后验均值基于后验方差，先验均值和方差，以及观察值和观察值方差确定
            post_mean = post_var * (prior_mean / prior_var + obs_case[l, t1] / obs_var)
            post_mean_rec[l,t] = post_mean

#             print("post_mean",np.sum(post_mean))
            # Compute alpha and adjust distribution to confirm to posterior moments
            # 计算α并调整分布以符合后验，通过观察方差和先验方差的开平方获得
            alpha = (obs_var / (obs_var + prior_var)) ** 0.5
            dy = post_mean + alpha * (obs_ens[l, :] - prior_mean) - obs_ens[l, :]
#             print("dy",np.sum(dy))
            # Loop over each state variable (connected to location l)
            # 循环处理每个状态变量（与位置l相连）
            # adjust related metapopulation，调整相关的metapopulation
            # E
            temp = E[l, :]
            # 计算估计的状态值与观察值之间的相关性，再除以先验方差
            A = np.cov(temp, obs_ens[l, :])
            E_A_rec[l,t] = A[1, 0]
            
#             print("E_A",E_A_rec[l,t])
            
            rr = A[1, 0] / prior_var
            dx = rr * dy
#             print("E",np.sum(E[l, :]))
            E[l, :] = E[l, :] + dx
#             print("E+dx",np.sum(E[l, :]))
            
            # Is
            temp = Is[l, :]
            A = np.cov(temp, obs_ens[l, :])
            Is_A_rec[l,t] = A[1, 0]
            rr = A[1, 0] / prior_var
            dx = rr * dy
            Is[l, :] = Is[l, :] + dx
            # Ia
            temp = Ia[l, :]
            A = np.cov(temp, obs_ens[l, :])
            Ia_A_rec[l,t] = A[1, 0]
            rr = A[1, 0] / prior_var
            dx = rr * dy
            Ia[l, :] = Ia[l, :] + dx
            # dailyIs
            temp = dailyIs_prior[l, :]
            A = np.cov(temp, obs_ens[l, :])
            dailyIs_A_rec[l,t] = A[1, 0]
#             print("dailyIs_A",dailyIs_A_rec[l,t])
            rr = A[1, 0] / prior_var
            dx = rr * dy
            # 保证值大于0
            dailyIs_prior[l, :] = np.round(np.maximum(dailyIs_prior[l, :] + dx, 0))
            # dailyIa
            temp = dailyIa_prior[l, :]
            A = np.cov(temp, obs_ens[l, :])
            dailyIa_A_rec[l,t] = A[1, 0]
            rr = A[1, 0] / prior_var
            dx = rr * dy
            dailyIa_prior[l, :] = np.round(np.maximum(dailyIa_prior[l, :] + dx, 0))

            # adjust alpha，alpha也采用同样的方式调整
            temp = para[amap[l], :]
            A = np.cov(temp, obs_ens[l, :])
            a_A_rec[l,t] = A[1, 0]
            rr = A[1, 0] / prior_var
            dx = rr * dy
            para[amap[l], :] = para[amap[l], :] + dx
            # inflation，加入膨胀因子
            if (np.std(para[amap[l], :] < parastd[amap[l]])):
                para[amap[l], :] = np.mean(para[amap[l], :]) * np.ones(num_ens) + lambda2 * (
                        para[amap[l], :] - np.mean(para[amap[l], :]) * np.ones(num_ens))

            # beta也用同样的方式调整，
            temp = para[Timap[l], :]
            A = np.cov(temp, obs_ens[l, :])
            Ti_A_rec[l,t] = A[1, 0]
            rr = A[1, 0] / prior_var
            dx = rr * dy
            para[Timap[l], :] = para[Timap[l], :] + dx
            # inflation
            if (np.std(para[Timap[l], :]) < parastd[Timap[l]]):
                para[Timap[l], :] = np.mean(para[Timap[l], :]) * np.ones(num_ens) + lambda2 * (
                        para[Timap[l], :] - np.mean(para[Timap[l], :]) * np.ones(num_ens))

        else:
            # adjust related metapopulation，还在调整延迟的循环中，调整相关metapopulation，令该天的不同状态为0
            #
            S[l, :] = popNew[l, t] * np.ones(num_ens)
            E[l, :] = 0
            Is[l, :] = 0
            Ia[l, :] = 0
            dailyIs_prior[l, :] = 0
            dailyIa_prior[l, :] = 0
    #     para = checkbound_para(para, paramax, paramin)
    for i in range(0, len(para)):
        para[i, para[i, :] < paramin[i]] = paramin[i] * (1 + 0.2 * np.random.rand(np.sum(para[i, :] < paramin[i])))
        para[i, para[i, :] > paramax[i]] = paramax[i] * (1 - 0.2 * np.random.rand(np.sum(para[i, :] > paramax[i])))
    # update posterior Is and Ia，更新后验Is和Ia，先令后验都等于先验
    dailyIs_post = dailyIs_prior
    dailyIa_post = dailyIa_prior

    cumu_dailyIs_post = cumu_dailyIs_post + dailyIs_post
    cumu_dailyIa_post = cumu_dailyIa_post + dailyIa_post
    
    #post之后是否一样
#     print('postSEIR-S',np.sum(S))
#     print('postSEIR-E',np.sum(E))
#     print('postSEIR-Is',np.sum(Is))   
#     print('postSEIR-Ia',np.sum(Ia))
#     print("dailyIs_post",np.sum(dailyIs_post))
#     print("dailyIa_post",np.sum(dailyIa_post))

    # 在9天延迟循环，月份仍然是对后几个月的调整
    if t < 116:
        monthnum = 0
    if ((t >= 116) & (t < 147)):
        monthnum = 1
    if ((t >= 147) & (t < 177)):
        monthnum = 2
    if ((t >= 177) & (t < 208)):
        monthnum = 3
    if ((t >= 208) & (t < 210)):
        monthnum = 4
    a = delaypara[monthnum, 0]
    b = delaypara[monthnum, 1]

    for k in range(0, num_ens):
        # update obs_temp
        
        for l in range(0, num_loc):        
            # 对于每个ensemble，如果有症状大于0，对报告进行延迟的随机采样
            inci = dailyIs_post[l, k].astype(int)
            
            if(inci > 0):
#                 print("ensemble",k)
#                 print("state",l)
#                 print("inci",inci)
                rnd = np.ceil(np.random.gamma(a,b,inci)).astype(int).tolist()
#                 print("rnd",rnd)
                index = np.array([l,t,k]).astype(int).tolist()
                rnds_all = index + rnd
                rnds_delay_tmp.append(rnds_all.copy())
                for h in range(0, len(rnd)):
                    if (t + rnd[h] < T):
                        obs_temp[l, k, t + rnd[h]] = obs_temp[l, k, t + rnd[h]] + 1
    #####update S
    S = popNew[:, t].reshape(num_loc, 1) * np.ones((1, num_ens)) - E - cumu_dailyIs_post - cumu_dailyIa_post
    ###############
    S, E, Is, Ia = checkbound(S, E, Is, Ia)
    


    # save posterior
    ###statevariables,存储后验分布
    # 即为每个位置每天每个ensemble的状态值
    for i in range(0, num_loc):
        for j in range(0, num_ens):
            S_post[i, t, j] = S[i, j] / popNew[i, t]
            E_post[i, t, j] = E[i, j] / popNew[i, t]
            Is_post[i, t, j] = Is[i, j] / popNew[i, t]
            Ia_post[i, t, j] = Ia[i, j] / popNew[i, t]
            dailyIs_post_rec[i, j, t] = dailyIs_post[i, j]
            dailyIa_post_rec[i, j, t] = dailyIa_post[i, j]

    para_post[:, :, t] = para

#计算程序耗时
end_time = time.time()
print("耗时: {:.2f}秒".format(end_time - start_time))
# para_post: posterior parameters
# S_post: posterior susceptibility in each county on each day
# dailyIs_post_rec: posterior estimate of the daily newly infected documented infections in each county on each day
# dailyIa_post_rec: posterior estimate of the daily newly infected undocumented infections in each county on each day
# obs_temp: posterior fitting of daily confirmed cases in each state





D:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
D:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


模拟天数 0
模拟天数 1
模拟天数 2
模拟天数 3
模拟天数 4
模拟天数 5
模拟天数 6
模拟天数 7
模拟天数 8
模拟天数 9
模拟天数 10
模拟天数 11
模拟天数 12
模拟天数 13
模拟天数 14
模拟天数 15
模拟天数 16
模拟天数 17
模拟天数 18
模拟天数 19
模拟天数 20
模拟天数 21
模拟天数 22
模拟天数 23
模拟天数 24
模拟天数 25
模拟天数 26
模拟天数 27
模拟天数 28
模拟天数 29
模拟天数 30
模拟天数 31
模拟天数 32
模拟天数 33
模拟天数 34
模拟天数 35
模拟天数 36
模拟天数 37
模拟天数 38
模拟天数 39
模拟天数 40
模拟天数 41
模拟天数 42
模拟天数 43
模拟天数 44
模拟天数 45
模拟天数 46
模拟天数 47
模拟天数 48
模拟天数 49
模拟天数 50
模拟天数 51
模拟天数 52
模拟天数 53
模拟天数 54
模拟天数 55
模拟天数 56
模拟天数 57
模拟天数 58
模拟天数 59
模拟天数 60
模拟天数 61
模拟天数 62
模拟天数 63
模拟天数 64
模拟天数 65
模拟天数 66
模拟天数 67
模拟天数 68
模拟天数 69
模拟天数 70
模拟天数 71
模拟天数 72
模拟天数 73
模拟天数 74
模拟天数 75
模拟天数 76
模拟天数 77
模拟天数 78
模拟天数 79
模拟天数 80
模拟天数 81
模拟天数 82
模拟天数 83
模拟天数 84
模拟天数 85
模拟天数 86
模拟天数 87
模拟天数 88
模拟天数 89
模拟天数 90
模拟天数 91
模拟天数 92
模拟天数 93
模拟天数 94
模拟天数 95
模拟天数 96
模拟天数 97
模拟天数 98
模拟天数 99
模拟天数 100
模拟天数 101
模拟天数 102
模拟天数 103
模拟天数 104
模拟天数 105
模拟天数 106
模拟天数 107
模拟天数 108
模拟天数 109
模拟天数 110
模拟天数 111
模拟天数 112
模拟天数 113
模拟天数 114
模拟天数 115
模拟天数 116
模拟天数 117
模拟天数 118
模拟天数 119
模拟天数 120
模拟天数 121
模拟天数 122
模拟天